In [1]:
import xray; print(xray.__version__)

0.5.0


### Reading data

In [2]:
infile = '/Users/damienirving/Downloads/Data/va_ERAInterim_500hPa_2006-030day-runmean_native.nc'
dset = xray.open_dataset(infile)

In [3]:
dset

<xray.Dataset>
Dimensions:    (latitude: 241, longitude: 480, time: 365)
Coordinates:
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 3.0 3.75 4.5 5.25 6.0 ...
  * latitude   (latitude) float32 -90.0 -89.25 -88.5 -87.75 -87.0 -86.25 ...
  * time       (time) datetime64[ns] 2006-01-01T06:00:00 2006-01-02T06:00:00 ...
Data variables:
    va         (time, latitude, longitude) float64 -3.818 -3.803 -3.787 ...
Attributes:
    CDI: Climate Data Interface version 1.5.3 (http://code.zmaw.de/projects/cdi)
    Conventions: CF-1.0
    history: Mon Jun 15 13:04:23 2015: ncatted -O -a axis,time,c,c,T va_ERAInterim_500hPa_2006-030day-runmean_native.nc
Mon Jun 15 13:04:23 2015: ncatted -O -a missing_value,va,o,f,-32767. va_ERAInterim_500hPa_2006-030day-runmean_native.nc
Mon Jun 15 13:03:30 2015: cdo seldate,2006-01-01,2006-12-31 va_ERAInterim_500hPa_030day-runmean_native.nc va_ERAInterim_500hPa_2006-030day-runmean_native.nc
Fri Mar 20 07:35:31 2015: ncatted -O -a axis,time,c,c,T /mnt/meteo0/dat

In [4]:
va_data = dset[['va']]
type(va_data)

xray.core.dataset.Dataset

In [5]:
va_data

<xray.Dataset>
Dimensions:    (latitude: 241, longitude: 480, time: 365)
Coordinates:
  * latitude   (latitude) float32 -90.0 -89.25 -88.5 -87.75 -87.0 -86.25 ...
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 3.0 3.75 4.5 5.25 6.0 ...
  * time       (time) datetime64[ns] 2006-01-01T06:00:00 2006-01-02T06:00:00 ...
Data variables:
    va         (time, latitude, longitude) float64 -3.818 -3.803 -3.787 ...
Attributes:
    CDI: Climate Data Interface version 1.5.3 (http://code.zmaw.de/projects/cdi)
    Conventions: CF-1.0
    history: Mon Jun 15 13:04:23 2015: ncatted -O -a axis,time,c,c,T va_ERAInterim_500hPa_2006-030day-runmean_native.nc
Mon Jun 15 13:04:23 2015: ncatted -O -a missing_value,va,o,f,-32767. va_ERAInterim_500hPa_2006-030day-runmean_native.nc
Mon Jun 15 13:03:30 2015: cdo seldate,2006-01-01,2006-12-31 va_ERAInterim_500hPa_030day-runmean_native.nc va_ERAInterim_500hPa_2006-030day-runmean_native.nc
Fri Mar 20 07:35:31 2015: ncatted -O -a axis,time,c,c,T /mnt/meteo0/dat

### Climatologies

You can group by `month`, `day`, `dayofyear`, `dayofweek` and whatever else belongs to the [`pandas.DatetimeIndex` class](http://pandas.pydata.org/pandas-docs/dev/generated/pandas.DatetimeIndex.html).

##### Monthly

In [6]:
mon_clim_va = va_data.groupby('time.month').mean('time')

In [7]:
mon_clim_va

<xray.Dataset>
Dimensions:    (latitude: 241, longitude: 480, month: 12)
Coordinates:
  * latitude   (latitude) float32 -90.0 -89.25 -88.5 -87.75 -87.0 -86.25 ...
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 3.0 3.75 4.5 5.25 6.0 ...
  * month      (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    va         (month, latitude, longitude) float64 -3.441 -3.431 -3.419 ...

#### Daily

The most obvious thing to do would be:

In [8]:
day_clim_va = va_data.groupby('time.dayofyear').mean('time')
day_clim_va

<xray.Dataset>
Dimensions:    (dayofyear: 365, latitude: 241, longitude: 480)
Coordinates:
  * latitude   (latitude) float32 -90.0 -89.25 -88.5 -87.75 -87.0 -86.25 ...
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 3.0 3.75 4.5 5.25 6.0 ...
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 ...
Data variables:
    va         (dayofyear, latitude, longitude) float64 -3.818 -3.803 -3.787 ...

However this is problematic when leap years are involved. Instead, but would be better to group by month and day, as [this](http://stackoverflow.com/questions/20973527/compute-daily-climatology-using-pandas-python) stack overflow questions suggests.

In [9]:
day_clim_va = va_data.groupby(['time.month', 'time.day']).mean('time')

AttributeError: 'list' object has no attribute 'ndim'

### Selection

It's possible to select a single dimension value or a slice of values:

In [10]:
va_data.sel(time=('2006-03-25'))

<xray.Dataset>
Dimensions:    (latitude: 241, longitude: 480, time: 1)
Coordinates:
  * latitude   (latitude) float32 -90.0 -89.25 -88.5 -87.75 -87.0 -86.25 ...
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 3.0 3.75 4.5 5.25 6.0 ...
  * time       (time) datetime64[ns] 2006-03-25T06:00:00
Data variables:
    va         (time, latitude, longitude) float64 -0.3191 -0.2874 -0.2556 ...
Attributes:
    CDI: Climate Data Interface version 1.5.3 (http://code.zmaw.de/projects/cdi)
    Conventions: CF-1.0
    history: Mon Jun 15 13:04:23 2015: ncatted -O -a axis,time,c,c,T va_ERAInterim_500hPa_2006-030day-runmean_native.nc
Mon Jun 15 13:04:23 2015: ncatted -O -a missing_value,va,o,f,-32767. va_ERAInterim_500hPa_2006-030day-runmean_native.nc
Mon Jun 15 13:03:30 2015: cdo seldate,2006-01-01,2006-12-31 va_ERAInterim_500hPa_030day-runmean_native.nc va_ERAInterim_500hPa_2006-030day-runmean_native.nc
Fri Mar 20 07:35:31 2015: ncatted -O -a axis,time,c,c,T /mnt/meteo0/data/simmonds/dbirving/ERA

In [11]:
va_data.sel(time=slice('2006-03-25', '2006-04-25'))

<xray.Dataset>
Dimensions:    (latitude: 241, longitude: 480, time: 32)
Coordinates:
  * latitude   (latitude) float32 -90.0 -89.25 -88.5 -87.75 -87.0 -86.25 ...
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 3.0 3.75 4.5 5.25 6.0 ...
  * time       (time) datetime64[ns] 2006-03-25T06:00:00 2006-03-26T06:00:00 ...
Data variables:
    va         (time, latitude, longitude) float64 -0.3191 -0.2874 -0.2556 ...
Attributes:
    CDI: Climate Data Interface version 1.5.3 (http://code.zmaw.de/projects/cdi)
    Conventions: CF-1.0
    history: Mon Jun 15 13:04:23 2015: ncatted -O -a axis,time,c,c,T va_ERAInterim_500hPa_2006-030day-runmean_native.nc
Mon Jun 15 13:04:23 2015: ncatted -O -a missing_value,va,o,f,-32767. va_ERAInterim_500hPa_2006-030day-runmean_native.nc
Mon Jun 15 13:03:30 2015: cdo seldate,2006-01-01,2006-12-31 va_ERAInterim_500hPa_030day-runmean_native.nc va_ERAInterim_500hPa_2006-030day-runmean_native.nc
Fri Mar 20 07:35:31 2015: ncatted -O -a axis,time,c,c,T /mnt/meteo0/d

For an arbitrary list of dimension values, you need to provide the exact date/time value (for single dates xray/pandas can guess what you meant).

In [12]:
va_data['time'][0:20]

<xray.DataArray 'time' (time: 20)>
array(['2006-01-01T17:00:00.000000000+1100',
       '2006-01-02T17:00:00.000000000+1100',
       '2006-01-03T17:00:00.000000000+1100',
       '2006-01-04T17:00:00.000000000+1100',
       '2006-01-05T17:00:00.000000000+1100',
       '2006-01-06T17:00:00.000000000+1100',
       '2006-01-07T17:00:00.000000000+1100',
       '2006-01-08T17:00:00.000000000+1100',
       '2006-01-09T17:00:00.000000000+1100',
       '2006-01-10T17:00:00.000000000+1100',
       '2006-01-11T17:00:00.000000000+1100',
       '2006-01-12T17:00:00.000000000+1100',
       '2006-01-13T17:00:00.000000000+1100',
       '2006-01-14T17:00:00.000000000+1100',
       '2006-01-15T17:00:00.000000000+1100',
       '2006-01-16T17:00:00.000000000+1100',
       '2006-01-17T17:00:00.000000000+1100',
       '2006-01-18T17:00:00.000000000+1100',
       '2006-01-19T17:00:00.000000000+1100',
       '2006-01-20T17:00:00.000000000+1100'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetim

In [13]:
va_data.sel(time=['2006-01-03T17:00:00.000000000+1100',
                  '2006-01-12T17:00:00.000000000+1100'])

ValueError: not all values found in index 'time'

#### Maths / stats

In [14]:
va_data.mean(dim='latitude')

<xray.Dataset>
Dimensions:    (longitude: 480, time: 365)
Coordinates:
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 3.0 3.75 4.5 5.25 6.0 ...
  * time       (time) datetime64[ns] 2006-01-01T06:00:00 2006-01-02T06:00:00 ...
Data variables:
    va         (time, longitude) float64 0.6413 0.5716 0.5214 0.4906 0.4691 ...

I need to figure out how to apply the pandas [moving (rolling) statistics](http://pandas.pydata.org/pandas-docs/stable/computation.html#moving-rolling-statistics-moments) to an `xray.DataArray`. [This page](http://xray.readthedocs.org/en/stable/pandas.html) talks about how I might do that.

### Writing to file

In [ ]:
va_data.to_netcdf(format='NETCDF3_CLASSIC')